# Gemini Vertex AI

Tahapan terakhir, digunakan model Gemini melalui Vertex AI untuk menghasilkan rekomendasi berdasarkan hasil keuangan pengguna. Proses ini diawali dengan autentikasi menggunakan file kredensial layanan Google Cloud agar dapat mengakses API Gemini. Selanjutnya, input ke model dikendalikan melalui sebuah file JSON yang telah disusun khusus, berisi format prompt, parameter model, dan struktur output yang diinginkan. Dengan pendekatan ini, sistem mampu memberikan saran finansial yang relevan dan kontekstual secara otomatis, berdasarkan pola dan indikator keuangan pengguna yang telah diolah sebelumnya.



# Package/Library yang digunakan

Pada tahapan pertama dari model Gemini Vertex AI, akan dilakukan import package/library yang diperlukan untuk kemudahan dalam proses autentikasi, pengelolaan proyek Google Cloud, serta pengiriman permintaan (request) ke model Gemini melalui Vertex AI.

*   `!pip install google-cloud-aiplatform`
    > Untuk berinteraksi dengan layanan Vertex AI di Google Cloud langsung dari notebook, termasuk mengakses model seperti Gemini, mengelola endpoint, dan mengirim permintaan prediksi

*   `import os`
    > Untuk berinteraksi dengan sistem operasi, seperti mengakses variabel lingkungan, mengatur path file, atau mengelola direktori. Dalam konteks ini, biasanya os digunakan untuk membaca file kredensial atau mengatur konfigurasi berbasis lingkungan secara dinamis.

*   `import vertexai`
    > Untuk menginisialisasi proyek dan lokasi dalam konteks pemanggilan model AI di Vertex serta untuk menyambungkan notebook ke layanan Vertex AI secara langsung.

*   `import json`
    > Untuk membaca, menulis, dan memproses data dalam format JSON serta membaca file JSON yang berisi prompt atau konfigurasi request yang akan dikirim ke model Gemini.

*   `from datetime import date`
    > Untuk mengelola tanggal saat ini juga untuk mencatat waktu permintaan, menandai data rekomendasi, atau menampilkan tanggal hasil evaluas

*   `from vertexai.preview.generative_models import GenerativeModel, GenerationConfig`
    > `GenerativeModel` digunakan untuk mengakses dan menggunakan model generatif seperti Gemini, sementara `GenerationConfig` digunakan untuk mengatur parameter seperti panjang output, jumlah respons, dan gaya jawaban model.


In [ ]:
!pip install google-cloud-aiplatform
!pip install google-generativeai

In [ ]:
import os
import vertexai
import json
from datetime import date

from vertexai.preview.generative_models import GenerativeModel, GenerationConfig
from google import genai
from google.genai import types
import base64

# Inisialisasi dan Pemanggilan Model Gemini

Tahapan selanjutnya adalah pemanggilan model AI yang telah dideploy melalui end point di Vertex AI.

Langkah pertama dilakukan yakni mengatur kredensial akses menggunakan perintah `os.environ["GOOGLE_APPLICATION_CREDENTIALS"]` untuk melakukan eutentikasi pada layanan Google Cloud secara aman.

Selanjutnya fungsi `generate()` digunakan untuk mengirim prompt berupa teks ke model. Dalam fungsi ini terdapat sebuah client `genai.Client()` yang diinisialisasi dengan paramater `vertexai=True`, project ID, serta lokasi region untuk memastikan permintaan diarahkan ke model yang sesuai.

Input teks dari pengguna selanjutnya dalam format `Part`menggunakan `types.Part.from_text(text=text)` kemudian dimasukkan ke dalam `content` dengan peran sebagai user. Konfigurasi untuk proses generasi ditetapkan melalui `GenerateContentConfig` yang mengatur beberapa parameter (`temperature`, `top_p` untuk mengontrol karakteris serta `max_output_tokens` untuk panjang output), serta `safety_settings` digunakan untuk memfilter keamanan yakni **mencegah** moderasi konten yang mengandung ujaran kebencian, konten yang dapat mendorong tindakan berbahaya, konten yang bersifat seksual eksplisit dan konten yang mengandung pelecehan.

Pemanggilan model dilakukan dengan menentukan endpoint yang spesifik melalui variabel `model` yang merujuk pada model costum di Vertex AI. Proses inferensi dilakukan dengan metode `generate_content_stream()`, digunakan untuk pengiriman respons secara bertahap (streaming). Setiap bagian dari respons yang diterima kemudian dicetak secara langsung, seperti proses percakapan.

In [ ]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/sakoo-459810-932f7c84db79.json"

def generate(text):
  client = genai.Client(
      vertexai=True,
      project="1024264061904",
      location="us-central1",
  )

  msg1_text1 = types.Part.from_text(text=text)

  contents = [
    types.Content(role="user", parts=[msg1_text1]),
  ]

  generate_content_config = types.GenerateContentConfig(
    temperature=1,
    top_p=0.95,
    max_output_tokens=8192,
    safety_settings=[
      types.SafetySetting(category="HARM_CATEGORY_HATE_SPEECH", threshold="OFF"),
      types.SafetySetting(category="HARM_CATEGORY_DANGEROUS_CONTENT", threshold="OFF"),
      types.SafetySetting(category="HARM_CATEGORY_SEXUALLY_EXPLICIT", threshold="OFF"),
      types.SafetySetting(category="HARM_CATEGORY_HARASSMENT", threshold="OFF"),
    ],
  )

  model = "projects/1024264061904/locations/us-central1/endpoints/8574516140443697152"

  for chunk in client.models.generate_content_stream(
    model=model,
    contents=contents,
    config=generate_content_config,
  ):
    print(chunk.text, end="")


# Testing Generate

Pada tahapan ini dilakukan testing untuk memeriksa hasil dari model terkait situasi keuangan user, memebrikan saran yang realistis dan bagaimana menberikan keputusan terkait prioritas va kondisi keuangan

In [ ]:
text_user="""
pendapatan bulanan sebesar Rp15.000.000, pengeluaran Rp5.000.000,
tabungan Rp5.000.000, cicilan bulanan Rp1.500.000, dan dana darurat Rp20.000.000,
serta total aset saya saat ini sebesar Rp50.000.000 dan total utang Rp1.000.000.
untuk keperluan membeli laptop high end
"""
generate(text_user)

Berikut rekomendasi keuangan Anda:

1. Kondisi Keuangan Sangat Sehat.
Dana darurat lebih dari cukup, utang hampir tidak ada, dan arus kas sangat positif.

2. Manfaatkan Tabungan dan Surplus Bulanan.
Gunakan tabungan sebagai DP dan surplus bulanan untuk cicilan laptop.

3. Dana Darurat Saat Ini Sudah Ideal.
Dana darurat Anda (Rp20.000.000) setara dengan 4 bulan pengeluaran (Rp5.000.000), sangat aman.

4. Langkah Konkret Selanjutnya.
Lakukan riset laptop high end yang diinginkan. Hitung cicilan bulanannya dari surplus yang ada. Jika memungkinkan, tambah sedikit dari tabungan untuk DP atau cicilan.